https://stackoverflow.com/questions/466790/assembly-code-vs-machine-code-vs-object-code
https://stackoverflow.com/questions/845355/do-programming-language-compilers-first-translate-to-assembly-or-directly-to-mac

# Object code, Machine code, C and ASM

To understand binaries, how they're made and with which process. We need to understand a few concepts.

## At the beginning was the Code.

At the beginning, there was nothing. The programmer created some C code.

It could have simple been something like this :

In [36]:
%%writefile ../code/empty_C_programm.c
int main(){return 0;}

Overwriting ../code/empty_C_programm.c


This is even simpler than an "Hello world!". But it's kinda useless so far...

## Then there was the ASM :

### With AT&T syntax :

Since the C code alone wasn't that useful, the programer decided to compile his programm using `gcc`. Not every compiler works the same, but `gcc` pre-compiles the C code into ASM. We can see the result of this operation his way :

In [42]:
%%script bash --no-raise-error
gcc -S -masm=att ../code/empty_C_programm.c -o ../code/att_empty_C_programm.s
cat ../code/att_empty_C_programm.s

	.file	"empty_C_programm.c"
	.text
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	movl	$0, %eax
	popq	%rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.ident	"GCC: (GNU) 13.2.1 20230801"
	.section	.note.GNU-stack,"",@progbits


Let's analyse the above code :

```asm
        .file    "empty_C_programm.c"
        .text
        .globl   main
        .type    main, @function
main:
.LF80:
        .cfi_startproc
        pushq    %rbp
        .cfi_def_cfa_offset 16
        .cfi_offset 6, -16
        .movq    %rsp, %rbp
        .cfi_def_cfa_register 6
        movl    $0, %eax
        popq    %rbp
        .cfi_endproc
.LFE0:
        .size   main, .-main
        .ident  "GCC: (GNU) 13.2.1 20230801"
        .section        .note.GNU-stack,"",@progbits
```

- L1 : `.file "empty_C_programm.c"` : 

### With Intel Syntax :

By default, gcc uses the AT&T syntax for ASM but it's also possible to get the Intel syntax :

In [39]:
%%script bash --no-raise-error
gcc -S -masm=intel ../code/empty_C_programm.c -o ../code/intel_empty_C_programm.s
cat ../code/intel_empty_C_programm.s

	.file	"empty_C_programm.c"
	.intel_syntax noprefix
	.text
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	mov	rbp, rsp
	.cfi_def_cfa_register 6
	mov	eax, 0
	pop	rbp
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.ident	"GCC: (GNU) 13.2.1 20230801"
	.section	.note.GNU-stack,"",@progbits


### With optimization :

`gcc` also provides a few optimization flags :

#### -O0 (default) :

Since this is the default, it's the equivalent of the above.

#### -Os (Optimize for size) :

In [38]:
%%script bash --no-raise-error
gcc -S -masm=intel ../code/empty_C_programm.c -Os -o ../code/Os_intel_empty_C_programm.s
cat ../code/Os_intel_empty_C_programm.s

	.file	"empty_C_programm.c"
	.intel_syntax noprefix
	.text
	.section	.text.startup,"ax",@progbits
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	xor	eax, eax
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.ident	"GCC: (GNU) 13.2.1 20230801"
	.section	.note.GNU-stack,"",@progbits


Here, the `xor eax, eax` is just a way to fill `eax` with `0` before return.

#### -Og (Optimize for debugging) :

In [ ]:
%%script bash --no-raise-error
gcc -S -masm=intel ../code/empty_C_programm.c -Og -g -o ../code/Og_intel_empty_C_programm.s
cat ../code/Og_intel_empty_C_programm.s

#### -O and -O1 :

In [ ]:
%%script bash --no-raise-error
gcc -S -masm=intel ../code/empty_C_programm.c -O1 -o ../code/O1_intel_empty_C_programm.s
cat ../code/O1_intel_empty_C_programm.s

#### -O2 :

In [ ]:
%%script bash --no-raise-error
gcc -S -masm=intel ../code/empty_C_programm.c -O2 -o ../code/O2_intel_empty_C_programm.s
cat ../code/O2_intel_empty_C_programm.s

#### -O3 :

In [29]:
%%script bash --no-raise-error
gcc -S -masm=intel ../code/empty_C_programm.c -O3 -o ../code/O3_intel_empty_C_programm.s
cat ../code/O3_intel_empty_C_programm.s

	.file	"empty_C_programm.c"
	.intel_syntax noprefix
	.text
	.section	.text.startup,"ax",@progbits
	.p2align 4
	.globl	main
	.type	main, @function
main:
.LFB0:
	.cfi_startproc
	xor	eax, eax
	ret
	.cfi_endproc
.LFE0:
	.size	main, .-main
	.ident	"GCC: (GNU) 13.2.1 20230801"
	.section	.note.GNU-stack,"",@progbits
